In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data = pd.read_csv("product_val.csv")

C:\Users\redrdes1\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (29,35,39,55,93,94,97,113,116,131,139,141,144,145,157,158,160,161,166,169,170,181,203,214,215,216,217,218) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data.dtypes

product_id                                              object
product_category_id                                    float64
product_brand_id                                        object
product_type_id                                        float64
product_profile_type                                   float64
business_unit                                          float64
is_vending                                             float64
survey_id                                              float64
vertical_id                                            float64
Vertical_1                                             float64
Vertical_2                                             float64
Vertical_3                                             float64
Vertical_4                                             float64
Vertical_5                                             float64
product_name                                            object
product_style                                          

In [8]:
data = data.set_index("product_id")

In [9]:
cols = list(data.columns)
[print(i) for i in range(len(cols)) if cols[i] == "AQL"]

21


[None]

In [10]:
data_ = data.iloc[:,21:]

In [11]:
cols = list(data_.columns)
num_cols = [col for col in cols if data_[col].dtypes == "float64"]

In [12]:
num_cols

['AQL',
 'Abraison',
 'Abrasion',
 'Abrasion (EN 530)',
 'Abrasion (EN)',
 'Air Permeability',
 'Anti-Static material (EN 1149-5)',
 'Anti-Static suit (ATEX)',
 'Anti-Static suit material',
 'Arc Flash',
 'BBP (ASTM F1670/F1671)',
 'BBP (ISO 16604)',
 'Back of Hand',
 'Back of Hand Impact Protection',
 'Bio Aerosol (ISO/DIS 22611)',
 'Blood-borne Pathogens (ASTM F1670/F1671)',
 'Blood-borne Pathogens (ISO 16604)',
 'Bloodborne Pathogens',
 'Breathability',
 'Breathablility',
 'Burst Strength (ASTM D3787)',
 'Burst strength (ASTM D751, D3787 or D774)',
 'Burst strength (NFPA)',
 'Chemical Degradation',
 'Chemical Permeation',
 'Chemical Resistance (EN), 15 chemicals',
 'Chemical Resistance (NFPA), 21 chemicals',
 'Chemical Resistance (NFPA), Warfare agents',
 'Chemical Resistance (NFPA),Warfare agents',
 'Cleanliness',
 'Cleanliness for Cleanroom',
 'Cold',
 'Cold Protection',
 'Comfort',
 'Contact Cold Protection',
 'Contact Heat ',
 'Contact Heat (in C)',
 'Contamination',
 'Convectiv

In [13]:
cols = list(data_.columns)

In [19]:
cat_cols = [col for col in cols if data_[col].dtypes == "object"]
cat_cols = cat_cols[:-1] #do not use dropped

data_cat = data_.loc[:,cat_cols]

data_cat = data_cat.fillna("No")

data_.drop(columns = "do not use",axis = 1, inplace = True)

data_ = pd.get_dummies(data_,columns = cat_cols,drop_first = True)

data_ = data_.fillna(0)

In [20]:
data_[num_cols]=data_[num_cols].apply(lambda x: (x-x.min())/(x.max()-x.min()))

data_ = data_.fillna(0)

In [22]:
train_1,test_1 = train_test_split(data_,test_size = 0.15,random_state = 42)
train_11,dev_11 = train_test_split(train_1,test_size = 0.15,random_state = 42)

In [23]:
encoding_dim = 10
input_data = Input(shape=(train_11.shape[1],))
encoded = Dense(encoding_dim,activation = "elu")(input_data)
decoded = Dense(train_11.shape[1],activation = "sigmoid")(encoded)
autoencoder = Model(input_data,decoded)
autoencoder.compile(optimizer = "adam",loss="mean_squared_error")
hist_auto = autoencoder.fit(train_11,train_11,epochs = 50,batch_size=128,shuffle=True,validation_data = (dev_11,dev_11))







Train on 7274 samples, validate on 1284 samples
Epoch 1/50
7274/7274 [==============================] - 1s 100us/step - loss: 0.2122 - val_loss: 0.1595
Epoch 2/50
7274/7274 [==============================] - 0s 35us/step - loss: 0.1123 - val_loss: 0.0765
Epoch 3/50
7274/7274 [==============================] - 0s 36us/step - loss: 0.0632 - val_loss: 0.0536
Epoch 4/50
7274/7274 [==============================] - 0s 51us/step - loss: 0.0492 - val_loss: 0.0457
Epoch 5/50
7274/7274 [==============================] - 0s 45us/step - loss: 0.0433 - val_loss: 0.0414
Epoch 6/50
7274/7274 [==============================] - 0s 36us/step - loss: 0.0397 - val_loss: 0.0385
Epoch 7/50
7274/7274 [==============================] - 0s 38us/step - loss: 0.0370 - val_loss: 0.0361
Epoch 8/50
7274/7274 [==============================] - 0s 36us/step - loss: 0.0346 - val_loss: 0.0338
Epoch 9/50
7274/7274 [==============================] - 0s 41us/step - loss: 0.0324 - val_loss: 0.0316
Epoch 10/50
7274/7

In [29]:
print(input_data)
print(encoded)
print(decoded)

Tensor("input_1:0", shape=(?, 232), dtype=float32)
Tensor("dense_1/Elu:0", shape=(?, 10), dtype=float32)
Tensor("dense_2/Sigmoid:0", shape=(?, 232), dtype=float32)


In [47]:
encoder = Model(input_data, encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
print(encoder)
print(encoded_input)
print(decoder_layer)
autoencoder.layers

Tensor("input_3:0", shape=(?, 10), dtype=float32)


In [40]:
decoder = Model(encoded_input, decoder_layer(encoded_input))
encoded_x = encoder.predict(test_1)
decoded_output = decoder.predict(encoded_x)

In [42]:
encoded_x.shape

(1511, 10)

In [43]:
encoded_all_data = encoder.predict(data_)
decoded_all_output = decoder.predict(encoded_x)

In [45]:
pd.DataFrame(encoded_all_data)

,0,1,2,3,4,5,6,7,8,9
0,-0.915201,0.773783,3.386075,5.728765,-0.915629,-0.897444,-0.895960,0.194201,-0.577511,-0.948862
1,-0.886571,6.468394,2.680238,4.343049,-0.895111,-0.937844,-0.945584,2.132584,-0.870767,-0.914994
2,-0.896195,0.964636,3.112709,5.305402,-0.888956,-0.898600,-0.898772,0.598130,-0.622376,-0.939899
3,-0.574223,0.883595,0.852774,0.827000,-0.590450,-0.573918,-0.600910,0.893718,-0.586782,-0.608760
4,-0.890730,5.084418,7.131347,2.963569,-0.698951,-0.930867,-0.907740,2.080511,0.157855,-0.892875
5,-0.962169,5.775103,6.215128,6.048031,-0.943723,-0.986805,-0.962517,2.438279,-0.616275,-0.965056
6,-0.668385,1.166517,1.227468,1.263511,-0.755575,-0.700653,-0.762270,1.342294,-0.619648,-0.674172
7,-0.943850,1.828549,3.431836,5.577243,-0.931243,-0.953880,-0.930476,0.274448,-0.742247,-0.938608
8,-0.859465,5.881209,3.808812,3.955250,-0.849273,-0.935338,-0.917929,1.746916,-0.785477,-0.905401
9,-0.861888,2.601315,4.562623,4.522810,-0.820177,-0.904235,-0.894232,0.712878,-0.352489,-0.931283


In [46]:
data_.shape

(10069, 232)